# Assignment 2 2024

In the first section of this assignment, your task is to perform a sentiment analysis task on the Twitter dataset, train.csv, provided using the BERT model. Your job is to answer each question carrying a particular mark. The BERT model can be imported from the huggingface library. Your codes have to be executable in Google Colab. The dataset contains around 32K tweets which are labeled based on having racist or sexist content.  

### Mount your google drive and and change your path

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/MachineLearning/Assignment 2')

### Install the huggingface module

In [3]:
!pip install transformers

### Install the preprocessing text

In [4]:
!pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.5 MB/s eta 0:00:00


### 1. Import all relevant modules here [5 marks].

##### place all relevant libraries into the following cell

In [51]:
import tensorflow as tf
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import re
from tensorflow.python.ops.numpy_ops import np_config

### 2. Load your dataset and show the first five rows of the dataset [5 marks].




##### you need to use Pandas in order to load the dataset

In [6]:
df = pd.read_csv("train.csv", encoding="latin-1")

df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


### 3. Given the following function to clean the raw text of your dataset, clean your dataset and show the first five rows of the clean dataset [5 marks].

##### apply the following function to clean your dataset.

In [7]:
# Cleaning Raw tweets
def clean_text(text):

    #remove emails
    text = ' '.join([i for i in text.split() if '@' not in i])

    #remove web address
    text = re.sub('http[s]?://\S+', '', text)

    #remove punctuations
    text = re.sub(r'[^\w\s]', '', text)

    #Filter to allow only alphabets
    text = re.sub(r'[^a-zA-Z0-9\']', ' ', text)

    #Remove Unicode characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)

    #remove double spaces
    text = re.sub('\s+', ' ', text)

    return text

In [8]:
df['tweet'] = df['tweet'].apply(clean_text)

df.head()



,id,label,tweet
0,1,0,when a father is dysfunctional and is so selfi...
1,2,0,thanks for lyft credit i cant use cause they d...
2,3,0,bihday your majesty
3,4,0,model i love u take with u all the time in ur
4,5,0,factsguide society now motivation


### 4. Preprocess your text dataset to be compatible to BERT model. Set MAX_LEN to 128 [20 marks].

##### the preprocesing step is to tokenize your dataset such that it is fit to input BERT

In [9]:

sentences = df['tweet'].values
sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

labels = df['label'].values

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("Tokenize the first sentence:")
print(tokenized_texts[0])

MAX_LEN = 128

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]


# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokenize the first sentence:
['[CLS]', 'when', 'a', 'father', 'is', 'dysfunction', '##al', 'and', 'is', 'so', 'selfish', 'he', 'drag', '##s', 'his', 'kids', 'into', 'his', 'dysfunction', 'run', '[SEP]']
[[  101  2043  1037 ...     0     0     0]
 [  101  4283  2005 ...     0     0     0]
 [  101 12170 14945 ...     0     0     0]
 ...
 [  101  5962  2000 ...     0     0     0]
 [  101 17246  3379 ...     0     0     0]
 [  101  4067  2017 ...     0     0     0]]


### 5. Split the dataset to the training set and the testing set with the ratio of 90% for the training dataset and 10% for the testing dataset [5 marks].

##### Both data and attention mask need to be split

In [10]:
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

### 6. Create the dataloader of the datasets with the batch size of 32 [5 marks].

##### The dataloader can be created in the tensorflow

In [11]:
batch_size = 32
epochs = 2

train_labels = tf.expand_dims(train_labels,axis=1)
validation_labels = tf.expand_dims(validation_labels, axis=1)

train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_masks, train_labels))
train_dataset = train_dataset.batch(batch_size)


validation_dataset = tf.data.Dataset.from_tensor_slices((validation_inputs, validation_masks, validation_labels))
validation_dataset = validation_dataset.batch(batch_size)

### 7. Create the BERT model from the hugging face library [5 marks].

##### You can craft your BERT manually or directly use a pre-developed model for text classification

In [12]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 8. Prepare the training environments for your BERT model with the cross entropy loss function and the ADAM optimizer [10 marks]

##### prepare your loss and learning rate

In [13]:
steps_per_epoch = len(train_dataset) // batch_size

warmup_steps = steps_per_epoch // 3

epochs = 2

loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

total_steps = len(train_dataset)*epochs - warmup_steps

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=2e-5, decay_steps=total_steps, decay_rate=0.96, staircase=False, name=None)

optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule, epsilon = 1e-8)

### 9. Train your BERT model with 2 epochs [20 marks].

##### you need to create your training loop here

In [14]:
def performance_metrics(preds, labels):
  pred_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  for gt, pred in zip(labels_flat, pred_flat):
    if gt == 1 and pred == 1:
      tp +=1
    elif gt==0 and pred == 0:
      tn += 1
    elif gt==0 and pred == 1:
      fp += 1
    else:
      fn += 1

  return tp, tn, fp, fn


In [ ]:
np_config.enable_numpy_behavior()


train_loss_set = []


for _ in trange(epochs, desc="Epoch"):


  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0


  for batch in tqdm(train_dataset):

    b_input_ids, b_input_mask, b_labels = batch

    with tf.device('/GPU:0'):

      with tf.GradientTape() as tape:

        logits = model(b_input_ids, b_input_mask)

        loss = loss_fn(b_labels, logits[0])

      gradients = tape.gradient(loss, model.trainable_variables)

      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss_set.append(loss)

    tr_loss += loss
    nb_tr_examples += b_input_ids.shape[0]
    nb_tr_steps +=1

  print("Train loss: {}".format(tr_loss/nb_tr_examples))


  eval_loss, eval_TP, eval_TN, eval_FP, eval_FN = 0, 0, 0, 0, 0

  for batch in validation_dataset:

    b_input_ids, b_input_mask, b_labels = batch

    with tf.device('/GPU:0'):

      logits = model(b_input_ids,b_input_mask,training=False)


    TP, TN, FP, FN = performance_metrics(logits[0].numpy(), b_labels.numpy())


    eval_TP += TP
    eval_TN += TN
    eval_FP += FP
    eval_FN += FN

  accuracy = (eval_TP + eval_TN)/ (eval_TP + eval_TN + eval_FP + eval_FN)
  precision = eval_TP / (eval_TP + eval_FP) if (eval_TP + eval_FP) != 0 else 0
  recall = eval_TP / (eval_TP + eval_FN) if (eval_TP + eval_FN) != 0 else 0
  F1_score = 2 * precision * recall/ (precision + recall) if (precision + recall) !=0 else 0

  print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {F1_score}")

100%|██████████| 899/899 [19:12<00:00,  1.28s/it]


Train loss: 0.014538583345711231


Epoch:  50%|█████     | 1/2 [20:06<20:06, 1206.23s/it]

Accuracy: 0.638723803565843, Precision: 0.09106830122591944, Recall: 0.47058823529411764, F1-Score: 0.15260454878943508



100%|██████████| 899/899 [19:21<00:00,  1.29s/it]


Train loss: 0.002915749792009592


Epoch: 100%|██████████| 2/2 [40:50<00:00, 1225.03s/it]

Accuracy: 0.030966531122927746, Precision: 0.017119838872104734, Recall: 0.23076923076923078, F1-Score: 0.031875


### 10. Evaluate your model with the testing dataset and calculate accuracy, precision, recall and F1 score [20 marks].

##### evaluation need to be done inside the training loop

In [ ]:
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1-Score: {F1_score}")

Accuracy: 0.5032843290584923, Precision: 0.12216694306246545, Recall: 1.0, F1-Score: 0.21773399014778325


In the second part of the assignment, you are suppossed to perform sentiment analysis on the smile-annotations-final.csv. It differs from the first part of the assignment where this dataset features a multi-class classification problem.

### 11. Load the dataset and show the first few rows of the dataset [5 marks].

##### you need to use Pandas to load the dataset

In [25]:
df_multi = pd.read_csv("smile-annotations-final.csv",encoding="latin-1")

df_multi.columns = ['Id', 'Text', 'Label']

df_multi.head()


,Id,Text,Label
0,614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,614877582664835073,@Sofabsports thank you for following me back. ...,happy
3,611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
4,611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy


### 12. Identify the number of classes in the dataset [5 marks].




##### you can identify the unique category in your dataset

In [43]:
num_classes = df_multi["Label"].nunique()
print(num_classes)

13


### 13. Form a six-classes classification problem that consists of happy, not-relevant, angry, surprise, sad and disgust [5 marks].

##### Remove irrelevant classes

In [45]:
target_classes = ["happy", "not-relevant", "angry", "surprise", "sad", "disgust"]
df_stripped = df_multi[df_multi["Label"].isin(target_classes)]
df_stripped = df_stripped[["Text", "Label"]]
print(df_stripped["Label"].value_counts())

Label
happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: count, dtype: int64


,Text,Label
0,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
1,@SelectShowcase @Tate_StIves ... Replace with ...,happy
2,@Sofabsports thank you for following me back. ...,happy
3,@britishmuseum @TudorHistory What a beautiful ...,happy
4,@NationalGallery @ThePoldarkian I have always ...,happy


### 14. Replace text labels with numeric labels [5 marks].

In [46]:
def label_mapping(text):
  if text == "happy":
    return 0
  elif text == "not-relevant":
    return 1
  elif text == "angry":
    return 2
  elif text == "surprise":
    return 3
  elif text == "sad":
    return 4
  elif text == "disgust":
    return 5

df_stripped["Label"] = df_stripped["Label"].apply(label_mapping)

df_stripped.head()

,Text,Label
0,Dorian Gray with Rainbow Scarf #LoveWins (from...,0
1,@SelectShowcase @Tate_StIves ... Replace with ...,0
2,@Sofabsports thank you for following me back. ...,0
3,@britishmuseum @TudorHistory What a beautiful ...,0
4,@NationalGallery @ThePoldarkian I have always ...,0


### 15. Remove irrelevant characters in the text and show the first few rows of the clean dataset [5 marks].

##### you can use the same function given in the first part of the assignment

In [48]:
df_stripped["Text"] = df_stripped["Text"].apply(clean_text)
df_stripped.head()

,Text,Label
0,Dorian Gray with Rainbow Scarf LoveWins from,0
1,Replace with your wish which the artist uses i...,0
2,thank you for following me back Great to hear ...,0
3,What a beautiful jewel portrait Is the R for Rex,0
4,I have always loved this painting,0


### 16. Preprocess your text dataset to be compatible to BERT model. Set MAX_LEN to 128 [20 marks].

##### tokenize your dataset such that it is ready for BERT to use

In [54]:
sentences = df_stripped.Text.values

sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]

labels = df_stripped.Label.values

one_hot_label = to_categorical(labels)

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print("Tokenize the first sentence:")
print(tokenized_texts[0])

MAX_LEN = 128

input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(input_ids)

Tokenize the first sentence:
['[CLS]', 'dorian', 'gray', 'with', 'rainbow', 'scarf', 'love', '##win', '##s', 'from', '[SEP]']
[[  101 16092  3897 ...     0     0     0]
 [  101  5672  2007 ...     0     0     0]
 [  101  4067  2017 ...     0     0     0]
 ...
 [  101 11047  2559 ...     0     0     0]
 [  101  2033  6491 ...     0     0     0]
 [  101  1996  3416 ...     0     0     0]]


### 17. Split the dataset to the training set and the testing set with the ratio of 90% for the training dataset and 10% for the testing dataset [5 marks].

##### Do not forget the attention mask to be split

In [55]:
attention_masks = []

for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)


train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, one_hot_label, random_state=2018, test_size=0.1)

train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

### 18. Create the dataloader of the datasets with the batch size of 32 [5 marks].

##### Dataloader can be created in the tensorflow

In [57]:
batch_size = 32
epochs = 2

train_labels = tf.expand_dims(train_labels,axis=1)
validation_labels = tf.expand_dims(validation_labels, axis=1)

train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_masks, train_labels))
train_dataset = train_dataset.batch(batch_size)

validation_dataset = tf.data.Dataset.from_tensor_slices((validation_inputs, validation_masks, validation_labels))
validation_dataset = validation_dataset.batch(batch_size)

### 19. Prepare the training environments for your BERT model with the cross entropy loss function and the ADAM optimizer [10 marks]

##### Prepare the loss function, the learning rate and the optimizer

In [59]:
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

steps_per_epoch = len(train_dataset) // batch_size

warmup_steps = steps_per_epoch // 3

loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False)

total_steps = len(train_dataset)*epochs - warmup_steps

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=2e-7, decay_steps=total_steps, decay_rate=0.96, staircase=False, name='ExponentialDecay')

optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule, epsilon = 1e-8)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 20. Train your BERT model with 2 epochs [20 marks].

##### Create your training loop

In [60]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis=1)
  labels_flat = np.argmax(labels, axis=1)
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
np_config.enable_numpy_behavior()

train_loss_set = []


for _ in trange(epochs, desc="Epoch"):

  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0


  for batch in tqdm(train_dataset):

    b_input_ids, b_input_mask, b_labels = batch

    with tf.device('/GPU:0'):

      with tf.GradientTape() as tape:

        logits = model(b_input_ids, b_input_mask)
        n1, n2 = tf.shape(logits[0])

        b_labels_reshape = tf.reshape(b_labels,(n1,n2))

        loss = loss_fn(b_labels_reshape, logits[0])

      gradients = tape.gradient(loss, model.trainable_variables)

      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss_set.append(loss)

    tr_loss += loss
    nb_tr_examples += b_input_ids.shape[0]
    nb_tr_steps +=1

  print("Train loss: {}".format(tr_loss/nb_tr_examples))

  for batch in validation_dataset:
    b_input_ids, b_input_mask, b_labels = batch

    with tf.device('/GPU:0'):

      logits = model(b_input_ids,b_input_mask,training=False)

      n1, n2 = tf.shape(logits[0])
      b_labels_reshape = tf.reshape(b_labels, (n1,n2))


  accuracy = flat_accuracy(logits[0].numpy(), b_labels_reshape.numpy())



 14%|█▍        | 6/42 [00:11<01:07,  1.87s/it]


### 21. Evaluate your model with the testing dataset and calculate accuracy [20 marks].

##### Evaluation needs to be done inside the training loop

In [62]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.09523809523809523
